In [ ]:
import requests
import os

# Define the API endpoint and your access key
url = "https://api.pexels.com/v1/search"
access_key = "MgxBjp4CnSFtTvWe3SeHtkTGp5oUOBWVG5S0PYZAd37hV3T3h2yY4PHp"

# Define the headers with your access key
headers = {
    "Authorization": access_key
}

# Define the path to the images folder
path = "./images"

# Define the number of images to download
num_images = 80

# Define the query parameters
query_params = {
    "query": "random", 
    "per_page": num_images
}

# Create the images folder if it doesn't exist
if not os.path.exists(path):
    os.makedirs(path)

# Send the API request and download the images
response = requests.get(url, headers=headers, params=query_params)
json = response.json()
for i in range(num_images):
    image_url = json["photos"][i]["src"]["original"]
    image_id = json["photos"][i]["id"]
    image_extension = ".jpg" # change extension as per your requirement
    image_filename = f"{image_id}{image_extension}"
    image_path = os.path.join(path, image_filename)

    # Download the image and save it to the images folder
    response = requests.get(image_url)
    with open(image_path, "wb") as f:
        f.write(response.content)


In [2]:
import os
import json
from PIL import Image
from PIL.ExifTags import TAGS

class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, bytes):
            try:
                return obj.decode('utf-8')
            except UnicodeDecodeError:
                return obj.decode('utf-8', 'replace')
        return json.JSONEncoder.default(self, obj)

# Path to the folder containing pictures
path_to_folder = './images'

# Create the metadata folder if it doesn't exist
if not os.path.exists('metadata'):
    os.mkdir('metadata')

# Loop over all pictures in the folder
for filename in os.listdir(path_to_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Open the image and get the EXIF data
        image = Image.open(os.path.join(path_to_folder, filename))
        exifdata = image.getexif()

        # Create a dictionary to store the metadata for this image
        metadata = {}

        # Loop over all EXIF tags and add them to the metadata dictionary
        for tag_id, value in exifdata.items():
            tag = TAGS.get(tag_id, tag_id)
            metadata[tag] = value

        # Write the metadata dictionary to a JSON file in the metadata folder
        with open(os.path.join('metadata', filename + '.json'), 'w') as f:
            json.dump(metadata, f, cls=MyEncoder)

/tmp/ipykernel_742870/175423086.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='./images/4399344.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_742870/175423086.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='./images/4321026.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_742870/175423086.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='./images/730981.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_742870/175423086.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='./images/675764.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_742870/175423086.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='./images/5699514.jpg'>
  image = Image.open(os.path.join(path_to_folder, filename))
/tmp/ipykernel_742870/175423086.py:26: ResourceWarning: unclosed file <_io.BufferedReader name='./imag

In [3]:
import os
import json
import torch
from PIL import Image
from PIL.ExifTags import TAGS
from transformers import ViTFeatureExtractor, ViTModel
import cv2
from colorthief import ColorThief
import pytesseract
from imageai.Detection import ObjectDetection
"""
# Download the YOLOv3 PyTorch model
url = "https://github.com/OlafenwaMoses/ImageAI/releases/download/essential-v4/pretrained-yolov3.h5"
response = requests.get(url)

with open("pretrained-yolov3.pt", "wb") as f:  
    f.write(response.content)

image_path = "./images"

# Initialize the ViT feature extractor and model
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTModel.from_pretrained("google/vit-base-patch16-224")

# Initialize ImageAI object detection model
detector = ObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("pretrained-yolov3.pt")
detector.loadModel()"""


import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn

# Initialize torchvision object detection model
model = fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

image_path = "./images"

# Function to get image metadata
def get_image_metadata(image_path):
    metadata = {}

    # Read the image using PIL and OpenCV
    pil_image = Image.open(image_path)
    cv_image = cv2.imread(image_path)

    # Get dominant color and color palette
    color_thief = ColorThief(image_path)
    dominant_color = color_thief.get_color(quality=1)
    color_palette = color_thief.get_palette(color_count=5)

    # Get orientation
    exif_data = pil_image._getexif()
    orientation = exif_data[274] if exif_data and 274 in exif_data else "unknown"


    # Get number of faces
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=3)
    num_faces = len(faces)

    # Get text
    pil_image_rgb = pil_image.convert('RGB')
    text = pytesseract.image_to_string(pil_image_rgb)

    # Get objects
    transform = T.Compose([T.Resize(256), T.CenterCrop(224), T.ToTensor()])
    img = transform(Image.open(image_path))
    img = img.unsqueeze(0)
    detections = model(img)

    objects = []
    for label, score in zip(detections[0]["labels"], detections[0]["scores"]):
        if score > 0.5:
            objects.append(label.item())


    # Build metadata dictionary
    metadata = {
        "orientation": orientation,
        "dominant_color": dominant_color,
        "color_palette": color_palette,
        "num_faces": num_faces,
        "text": text,
        "objects": objects,
    }

    return metadata

# Loop over all pictures in the folder
for filename in os.listdir(path_to_folder):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
        # Get the image metadata
        image_path = os.path.join(path_to_folder, filename)
        metadata = get_image_metadata(image_path)

        # Write the metadata dictionary to a JSON file in the metadata folder
        with open(os.path.join("metadata", filename + ".json"), "w") as f:
            json.dump(metadata, f)




/tmp/ipykernel_742870/2965986020.py:71: ResourceWarning: unclosed file <_io.BufferedReader name='./images/4399344.jpg'>
  img = transform(Image.open(image_path))
